<a href="https://colab.research.google.com/github/LeeSeungwon89/Python_for_Data_Analytics_Science/blob/main/4.%20%EB%8D%B0%EC%9D%B4%ED%84%B0%20%EC%A0%84%EC%B2%98%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **1. 데이터 전처리하기**

각종 인터넷 정보, 서적, **파이썬 머신러닝 판다스 데이터 분석(오승환, 정보문화사)**를 참고했습니다.

## **1.1. 누락된 데이터**

특정 데이터를 실수로 입력하지 않거나 파일 형식을 변환하면서 데이터 자체가 손상되는 경우가 많습니다. 누락된 데이터를 적절하게 처리해야 데이터 분석 결과가 왜곡되지 않습니다.

### **1.1.1 누락된 데이터 확인하기**

### **1.1.2. 누락된 데이터 제거하기**

### **1.1.3. 누락된 데이터 치환하기**

## **1.2. 중복된 데이터**

### **1.2.1. 중복된 데이터 확인하기**

### **1.2.2. 중복된 데이터 제거하기**

## **1.3. 데이터 표준화**

### **1.3.1. 단위 환산하기**

### **1.3.2. 자료형 변환하기**

## **1.4. 범주형 데이터**

### **1.4.1. 구간 분할하기**

### **1.4.2. 더미 변수 사용하기**

## **1.5. 데이터 정규화**

## **1.6. 시계열 데이터**

### **1.6.1. 시계열 객체로 변환하기**

### **1.6.2. 시계열 데이터 생성하기**

### **1.6.3. 시계열 데이터 활용하기**